In [1]:
import numpy as np
from  user import *
from  item import *
from user_based_Recommendor import *
from item_based_Recommendor import *

In [2]:
#there are 943 users and 1682 items
user_item_matrix=np.full((943+1,1682+1),-1) #each row represents a user and each col represents a item, user_item_matrix[i][j]=user i's rating of item j
file = open('ml-100k/u.data', 'r')
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('\t')
    user_id=int(splits[0])
    item_id=int(splits[1])
    rating=int(splits[2])

    user_item_matrix[user_id][item_id]=rating

In [3]:
#there are 943 users and 1682 items
item_user_matrix=np.full((1682+1,943+1),-1) #each row represents a user and each col represents a item, user_item_matrix[i][j]=user i's rating of item j
file = open('ml-100k/u.data', 'r')
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('\t')
    user_id=int(splits[0])
    item_id=int(splits[1])
    rating=int(splits[2])

    item_user_matrix[item_id][user_id]=rating

In [4]:
user_dict={}
file = open('ml-100k/u.user', 'r')
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('|')
    user_id=splits[0]

    new_user=user(user_id,splits[1],splits[2],splits[3],splits[4])#id,age,gender,occupation,zip_code
    user_dict[user_id]=new_user


In [5]:
print(user_dict['1'].printUserInfor())

User id =1, age=24, gender=M, occupation=technician, zip code=85711
None


In [6]:
item_dict={}
file = open('ml-100k/u.item', 'r',encoding="ISO-8859-1")
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('|')#1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
    movie_id=splits[0]
    new_movie=movie(item_id,splits[1],splits[2],splits[3],splits[4],getTypes(line[-38:]))
    item_dict[movie_id]=new_movie

In [7]:
print(item_dict['1'].printMovieInfor())

movie id=203
movie title=Toy Story (1995)
release date=01-Jan-1995
video release date=
IMDB URL=http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
type=Animation,Children's,Comedy
None


In [18]:
user_based_Model_Test(user_item_matrix,3)

  0%|          | 0/2 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [10]:
#item_based_Model_Test(item_user_matrix,3)

100%|██████████| 2/2 [00:59<00:00, 29.87s/it]RMSE=1.4295393602103366

